# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125842e+02     1.553465e+00
 * time: 0.4762899875640869
     1     1.189977e+01     9.194672e-01
 * time: 1.1469368934631348
     2    -1.143717e+01     1.103158e+00
 * time: 1.2140679359436035
     3    -3.349613e+01     7.837446e-01
 * time: 1.3096158504486084
     4    -4.733384e+01     5.177468e-01
 * time: 1.4049098491668701
     5    -5.660787e+01     1.850758e-01
 * time: 1.5060808658599854
     6    -5.961684e+01     1.121069e-01
 * time: 1.5640180110931396
     7    -6.080106e+01     4.922414e-02
 * time: 1.6215238571166992
     8    -6.125447e+01     6.446105e-02
 * time: 1.680351972579956
     9    -6.157036e+01     4.066109e-02
 * time: 1.7380950450897217
    10    -6.180849e+01     2.785201e-02
 * time: 1.8163518905639648
    11    -6.198834e+01     2.284644e-02
 * time: 1.8762099742889404
    12    -6.206086e+01     1.901644e-02
 * time: 1.9356269836425781
    13    -6.213305e+01     1.339460e-02
 * time: 1.99501585960

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671067
    AtomicLocal         -18.8557574
    AtomicNonlocal      14.8522555
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485354 
    Xc                  -19.3336815

    total               -62.261666457516